# Modified Multiplication Table Method

## Definir fonction

fonction objective

In [1]:
import sympy as sp
import numpy as np
import math


In [2]:
global variables
variables=[]
variables=[]
dictionaire_auxiliary_variable={}
res=1
variables_Ising =[]
dict_variable_model_map={}

def variable_name(n,prefix):  # np est len de p en binaire, nq est len de q en binaire
    var_name_list = []
    for i in range(n):
        var_name_list.append(prefix+str(i+1))
    return var_name_list

def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

def create_varibales_from_list(list):
    for element in list:
        create_variables(element)


In [3]:
p_varname = variable_name(4,'p')
q_varname = variable_name(4,'q')
c_varname = variable_name(4,'c')

create_varibales_from_list(p_varname)
create_varibales_from_list(q_varname)
create_varibales_from_list(c_varname)

In [4]:
expr = (
    (2 * sp.Symbol('p2') + 2 * sp.Symbol('p1') * sp.Symbol('q1') + 2 * sp.Symbol('q2')
     - 8 * sp.Symbol('c2') - 4 * sp.Symbol('c1') + sp.Symbol('p1') + sp.Symbol('q1') - 3)**2
    + (2 * sp.Symbol('q1') + 2 * sp.Symbol('p2') * sp.Symbol('q2') + 2 * sp.Symbol('p1')
       + 2 * sp.Symbol('c2') - 8 * sp.Symbol('c4') - 4 * sp.Symbol('c3')
       + sp.Symbol('p2') * sp.Symbol('q1') + sp.Symbol('p1') * sp.Symbol('q2')
       + sp.Symbol('c1') + 1)**2
    + (sp.Symbol('q2') + sp.Symbol('p2') + sp.Symbol('c3') + 2 * sp.Symbol('c4') - 2)**2
)

expr

(c3 + 2*c4 + p2 + q2 - 2)**2 + (-4*c1 - 8*c2 + 2*p1*q1 + p1 + 2*p2 + q1 + 2*q2 - 3)**2 + (c1 + 2*c2 - 4*c3 - 8*c4 + p1*q2 + 2*p1 + p2*q1 + 2*p2*q2 + 2*q1 + 1)**2

In [5]:
def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

# les varaiables sont sort par la lettre puis un nombre: "x12" -> "x" 12
def sort_key(variable):
    name = str(variable)
    # Split le nom du variable aux deux parties: letter & number
    letter = name[0]
    number = int(name[1:])
    return (letter, number)

def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term

# si #variables > 2 : diemension -1
def reduct_demension_term(coef,variables_liste):
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]

    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    rest_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    #check dict s'il existe déjà une variable auxiliary qui = les deux meme variable
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:#sinon, il faut creer une variable auxiliary
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1
    else:
        x_4 = key

    expr_reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = rest_expr * expr_reduct_formula
    return sp.expand(expr_reduct)

def reduct_demension_expression(expr):
    new_expr = 0
    need_reduction = False

    for term in expr.args:
        # get coefficant(la constante) et variables(dans une liste)
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            if len(variables_liste) > 3 :
                need_reduction = True
            new_term = reduct_demension_term(coef, variables_liste)
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction: # récursive
        return reduct_demension_expression(new_expr)
    else:
        return new_expr

In [6]:
expr_reduct_power = reduct_variable_power_function(expr)
reduct_dimension_expr = reduct_demension_expression(expr_reduct_power)

In [7]:
def transfers_qubo2Ising(reduct_demension_expr):
    subs_relations = {(variable, ( variable + 1 ) /2) for variable in variables}

    # Après la substitution, variables dans l'expression sont des variables d'Ising dans {-1,+1}
    new_expr = reduct_demension_expr.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr

expr_ising = transfers_qubo2Ising(reduct_dimension_expr)

import dimod
from dwave.preprocessing.composites import SpinReversalTransformComposite

def get_solver_parameter(expr_ising):
    linear = {}
    quadratic ={}
    offset = 0

    for term in expr_ising.args:
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]
        if len(term.free_symbols) ==0:
            offset = coef
        if len(term.free_symbols) ==1:
            linear[variables_liste[0].name] = coef
        elif len(term.free_symbols) ==2:
            quadratic[(variables_liste[0].name,variables_liste[1].name)] = coef

    return (linear,quadratic,offset)

(linear,quadratic,offset) = get_solver_parameter(expr_ising)

In [8]:
print(linear)

{'x2': -60, 'x1': -35, 'x4': -30, 'x5': -20, 'x12': -10, 'x13': -10, 'x6': -10, 'x10': 15, 'x11': 15, 'x8': 15, 'c4': 45, 'c2': 53, 'q1': -135/4, 'x3': -35/2, 'q2': -5/4, 'x14': 5/2, 'p2': 13/4, 'x7': 15/2, 'p1': 39/4, 'c3': 45/2, 'c1': 53/2, 'x9': 69/2}


In [9]:
quadratic

{('c4', 'q2'): 1,
 ('q1', 'x8'): 1,
 ('q2', 'x1'): 1,
 ('q2', 'x7'): 1,
 ('c3', 'q2'): 1/2,
 ('p2', 'q2'): 1/2,
 ('q1', 'x7'): 1/2,
 ('q2', 'x3'): 1/2,
 ('x9', 'x14'): 1/2,
 ('c1', 'p2'): -1,
 ('c1', 'q1'): -1,
 ('p1', 'x9'): -28,
 ('p2', 'x9'): -28,
 ('c4', 'p2'): -23,
 ('c2', 'p1'): -16,
 ('c4', 'p1'): -16,
 ('c2', 'x8'): -12,
 ('p1', 'x10'): -12,
 ('p2', 'x11'): -12,
 ('p2', 'x8'): -12,
 ('q1', 'x10'): -12,
 ('q1', 'x11'): -12,
 ('c1', 'p1'): -8,
 ('c2', 'c4'): -8,
 ('c2', 'q2'): -8,
 ('c3', 'p1'): -8,
 ('c4', 'q1'): -8,
 ('q1', 'x1'): -8,
 ('q2', 'x2'): -8,
 ('c1', 'x7'): -6,
 ('p2', 'x7'): -6,
 ('c1', 'c4'): -4,
 ('c1', 'q2'): -4,
 ('c2', 'c3'): -4,
 ('c3', 'q1'): -4,
 ('q1', 'x2'): -4,
 ('q1', 'x3'): -4,
 ('q2', 'x4'): -4,
 ('q2', 'x5'): -4,
 ('c1', 'c3'): -2,
 ('c2', 'p2'): -2,
 ('c2', 'q1'): -2,
 ('q1', 'x14'): -2,
 ('q1', 'x4'): -2,
 ('q2', 'x14'): -2,
 ('q2', 'x6'): -2,
 ('x9', 'x12'): -2,
 ('x9', 'x13'): -2,
 ('q1', 'q2'): 2,
 ('q2', 'x8'): 2,
 ('q1', 'x9'): 3,
 ('q2', 'x10'